# Imports

In [1]:
import jsonlines
import time

from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoTokenizer

/Users/danmueller/Documents/GitHub/aerospace_chatbot/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/danmueller/Documents/GitHub/aerospace_chatbot/.venv/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [15]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

# os.getenv('HUGGINGFACE_TOKEN')

True

# Read training data

In [ ]:
data_folder='../data/AMS/'
model='TheBloke/'
name='Llama-2-7B-Chat-GGUF'
file=data_folder+'ams_data_train_'+name+'-100.jsonl'

training_data = []
with jsonlines.open(file) as reader:
    for line in reader:
        training_data.append(line)

In [ ]:
dataset=load_dataset('dsmueller/ams_data_train_Llama-2-7B-Chat-GGUF-LLM-generic_100',split='train')

In [ ]:
dataset[0]['text']

# Using SFFTrainer
https://huggingface.co/docs/trl/sft_trainer 

## Example straight from https://huggingface.co/docs/trl/v0.7.4/en/sft_trainer

In [ ]:
dataset = load_dataset("imdb", split="train")

trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
)
trainer.train()

## My data

In [ ]:
model_name='TinyLlama/TinyLlama-1.1B-Chat-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokens=[]
for data in dataset:
    tokens.append(len(tokenizer.tokenize(data['text'])))
print(max(tokens))

In [ ]:
start_time=time.time()

trainer = SFTTrainer(
    model_name,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max(tokens),
)
trainer.train()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"The command took {elapsed_time:.4f} seconds to execute.")

Problem with not enough memory. Not likely to succeed here running locally on a mac on decent sized models.

# Using autotrain-advanced
* https://github.com/huggingface/autotrain-advanced 
* https://www.youtube.com/watch?v=3fsn19OI_C8&ab_channel=AbhishekThakur 

In [18]:
!source ../.venv/bin/activate
# !autotrain llm --train --project_name my-llm --model TinyLlama/TinyLlama-1.1B-Chat-v0.1 --data_path . --use-peft --use_int4 --learning_rate 2e-4 --train_batch_size 6 --num_train_epochs 3 --trainer sft

# The training dataset to be used must be called training.csv and be located in the data_path folder.
!autotrain llm --train \
    --project_name ./llms/ams_data_train_Llama-2-7B-Chat-GGUF-LLM-generic-100 \
    --model TinyLlama/TinyLlama-1.1B-Chat-v0.1 \
    --data_path . \
    --use-peft \
    --learning_rate 2e-4 \
    --train_batch_size 6 \
    --num_train_epochs 3 \
    --trainer sft \
    --push_to_hub \
    --repo_id ai-aerospace/ams_data_train_Llama-2-7B-Chat-GGUF-LLM-generic-100 \
    --token $HUGGINGFACE_TOKEN

⚠️ WARNING | 2023-12-10 18:47:52 | autotrain.cli.run_dreambooth:<module>:14 - ❌ Some DreamBooth components are missing! Please run `autotrain setup` to install it. Ignore this warning if you are not using DreamBooth or running `autotrain setup` already.
/Users/danmueller/Documents/GitHub/aerospace_chatbot/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
/Users/danmueller/Documents/GitHub/aerospace_chatbot/.venv/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
> INFO    Running LLM
> INFO    Params: Namespace(version=False, train